In [ ]:
import pandas as pd
import numpy as np
from zipline.api import order_target_percent, schedule_function, date_rules, time_rules, record, symbol
from zipline import run_algorithm
from zipline.assets.synthetic import make_simple_equity_info
from zipline.data.data_portal import DataPortal
# from zipline.data.treasury import load_treasury_curves
from zipline.data.loader import USEquityPricingLoader
from zipline.data.us_equity_pricing import USEquityPricing
from zipline.pipeline.loaders.frame import DataFrameLoader
from zipline.utils.calendar_utils import get_calendar
from zipline.assets import AssetFinder, Asset
from datetime import datetime

# === Load and prepare your DataFrame ===
file_path = r'..\Data\OPCL_20000103_20201231.csv'
df = pd.read_csv(file_path)
df.set_index('ticker', inplace=True)
df.columns = pd.to_datetime(df.columns.str.lstrip('X'), format='%Y%m%d')
df = df.transpose()
df.index.name = 'date'
df = df.astype(float)
df.replace(0, np.nan, inplace=True)
df = df.ffill()

# === Create fake OHLCV from 'close' data ===
assets = df.columns.tolist()
ohlcv_dict = {}
for col in ['open', 'high', 'low', 'close', 'volume']:
    if col == 'volume':
        ohlcv_dict[col] = pd.concat([pd.Series(1e6, index=df.index, name=asset) for asset in assets], axis=1)
    else:
        ohlcv_dict[col] = df.copy()
    ohlcv_dict[col].columns.name = 'asset'

# === Define asset metadata ===
metadata = make_simple_equity_info(
    symbols=assets,
    start_date=df.index.min(),
    end_date=df.index.max(),
    exchange='NYSE'
)
asset_finder = AssetFinder(metadata=metadata)

# Map symbols to Asset objects
symbols_to_assets = {s: asset_finder.lookup_symbol(s, as_of_date=df.index[0]) for s in assets}

# === Create DataFrameLoader for OHLCV ===
loaders = {
    USEquityPricing.open: DataFrameLoader(USEquityPricing.open, ohlcv_dict['open']),
    USEquityPricing.high: DataFrameLoader(USEquityPricing.high, ohlcv_dict['high']),
    USEquityPricing.low: DataFrameLoader(USEquityPricing.low, ohlcv_dict['low']),
    USEquityPricing.close: DataFrameLoader(USEquityPricing.close, ohlcv_dict['close']),
    USEquityPricing.volume: DataFrameLoader(USEquityPricing.volume, ohlcv_dict['volume']),
}

trading_calendar = get_calendar("XNYS")

# === Create a DataPortal ===
data_portal = DataPortal(
    asset_finder=asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=df.index.min(),
    equity_daily_reader=None,  # Not used
    adjustment_reader=None,    # No splits/dividends
    equity_loader=lambda column: loaders[column],
    # treasury_curves=load_treasury_curves(),
)

# === Define Strategy ===
def initialize(context):
    context.assets = list(symbols_to_assets.values())
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open())

def rebalance(context, data):
    weight = 1.0 / len(context.assets)
    for asset in context.assets:
        if data.can_trade(asset) and data.current(asset, 'price') > 1e-6:
            order_target_percent(asset, weight)
        else:
            order_target_percent(asset, 0)

def handle_data(context, data):
    record(**{asset.symbol: data.current(asset, 'price') for asset in context.assets[:3]})

# === Run Backtest ===
results = run_algorithm(
    start=df.index.min().tz_localize('UTC'),
    end=df.index.max().tz_localize('UTC'),
    initialize=initialize,
    handle_data=handle_data,
    capital_base=1e6,
    data_frequency='daily',
    data_portal=data_portal,
    trading_calendar=trading_calendar,
)

print(results.tail())


ModuleNotFoundError: No module named 'zipline.utils.calendars'